In [27]:
import pandas as pd
from itertools import permutations
import requests
from bs4 import BeautifulSoup
import json
# import geopy.distance
# from shapely.geometry import Point
# import geopandas as gpd
# from geopandas import GeoDataFrame
# import fiona
# from currency_converter import CurrencyConverter
# from bokeh.plotting import figure, save, show
# from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
# from pathlib import Path

In [28]:
#converter = CurrencyConverter()
#DESIRED_CUR = 'USD'

In [29]:
base_url = 'https://www.rome2rio.com/map/'
# airports_file_path = "../files/csv/Locations with airports.csv"
output_path = "../files/output/csv_output/"

In [30]:
def get_url(url, english=True):
    if english:
        r = requests.get(url, headers={'Accept-Language': 'en-US,en;q=0.5'})
    else:
        r = requests.get(url)

    if r.status_code != 200:
        print("Invalid page!")
        return []
    else:
        data = {}
        soup = BeautifulSoup(r.content, 'html.parser')
        # data["title"] = soup.find('h1').text
        dis = soup.find('meta', {'id': 'deeplinkTrip'})
        parsed = json.loads(dis["content"])[2][1]

        # print(json.dumps(parsed, indent=4, sort_keys=True))
        return parsed


class Cities:
    def __init__(self):
        #self.path = cities_file
        self.base_url = 'https://www.rome2rio.com/map/'
        try:
            #self.df = pd.read_csv(filename)
            pass
        except FileNotFoundError:
            print("File Not Found")
        self.routes = {}
        # self.pairs = []

    """ def gen_combinations(self, n=2):
        self.pairs = permutations(self.df['city'], n) """
        
    """ def get_routes(self, city1, city2):
        pass """

    def scrap_routes(self, city1, city2):
        tmp_url = self.base_url + city1 + '/' + city2
        self.routes[(city1, city2)] = get_url(tmp_url)


In [31]:
from_city, to_city = 'Kraków', 'Shanghai'

In [32]:
# attempt to extract elementary routes for each path
c = Cities()
c.scrap_routes(from_city, to_city)
pathes = c.routes[(from_city, to_city)]

In [33]:
def get_bb_id(coords):
    pass
    return -1

In [34]:
def get_aiport_id(code):
    pass
    return -1

In [35]:
""" 1-level - avaliable pathes
    2-level  - path summary - path[8] 
    
    TRAIN, SUBWAY, BUS, BUSFERRY, CAR, CARFERRY, CARTRAIN, FERRY, RIDESHARE, TRAM
        *   estimate_price[] = [i][8][j][13]
        *   duration, sec = [i][8][j][3]
        *   vehicle_type = [i][8][j][1]
        *   distance, km = [i][8][j][5]
        *   start_point[] = [i][8][j][6]
        *   end_point[] = [i][8][j][7]
        *   all_points[] = [i][8][j][18]
    FLY
        *   estimate_price[] = [i][8][j][11]
        *   duration, sec = [i][8][j][4]
        *   vehicle_type [flight] = [i][8][j][0]
        *   distance = [i][8][j][]
        *   start_point[] = [i][8][j][2]
        *   end_point[] = [i][8][j][3]
        *   all_points[] = [i][8][j][18] 
        *   aiport code = [i][8][j][2] 
"""
        
""" for i, item in enumerate(pathes[0][8][2]):
    print(i, '\n\t', item) """

" for i, item in enumerate(pathes[0][8][2]):\n    print(i, '\n\t', item) "

In [36]:
output_file = output_path + f'{from_city}-{to_city}' + '.csv'

data = {'from':[], 'from_bb_id':[], 
        'to':[], 'to_bb_id':[], 
        'transport':[], 
        'from_airport':[], 
        'from_airport_id':[],
        'to_airport':[], 
        'to_airport_id':[],
        'price':[], 'distance':[], 'duration':[], 'path_num':[]}

for path_num, path in enumerate(pathes):
    for route_num, route in enumerate(path[8][:-1]):
        if route[0] in ('walk', 'hotel'):
            continue
        if route[0] == 'flight':
            data['from'].append(route[2][1])
            data['from_bb_id'].append(get_bb_id(route[2][2:4]))
            data['to'].append(route[3][1])
            data['to_bb_id'].append(get_bb_id(route[3][2:4]))
            data['transport'].append(route[0])
            data['from_airport'].append(route[2][0])
            data['from_airport_id'].append(get_aiport_id(route[2][0]))
            data['to_airport'].append(route[3][0])
            data['to_airport_id'].append(get_aiport_id(route[3][0]))
            data['price'].append(route[11][0][2])
            data['distance'].append('')
            data['duration'].append(route[4])
            data['path_num'].append(path_num)
        else:
            data['from'].append(route[6][1])
            data['from_bb_id'].append(get_bb_id(route[6][2:4]))
            data['to'].append(route[7][1])
            data['to_bb_id'].append(get_bb_id(route[7][2:4]))
            data['transport'].append(route[1])
            data['from_airport'].append('')
            data['to_airport'].append('')
            data['from_airport_id'].append('')
            data['to_airport_id'].append('')
            data['price'].append(route[13][0][2])
            data['distance'].append(route[5])
            data['duration'].append(route[3])
            data['path_num'].append(path_num)
                            
tmp_df = pd.DataFrame(data)

# uncomment line below to make output in folder "../files/output/csv_output/"
tmp_df.to_csv(output_file) 

print(tmp_df)

                            from  from_bb_id                            to  \
0                  Kraków Główny          -1               Kraków Lotnisko   
1                         Krakow          -1               Shanghai Pudong   
2   Pudong International Airport          -1                 Longyang Road   
3                  Longyang Road          -1             West Nanjing Road   
4                  Kraków Główny          -1               Kraków Lotnisko   
5                         Krakow          -1             Shanghai Hongqiao   
6            Hongqiao Airport T1          -1            South Shaanxi Road   
7                         Kraków          -1                      Budapest   
8              Keleti pályaudvar          -1               Deák Ferenc tér   
9              Deák Ferenc tér M          -1        Liszt Ferenc Airport 2   
10                      Budapest          -1               Shanghai Pudong   
11  Pudong International Airport          -1                 Lon